In [1]:
import sys
print(sys.version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]


# Employment Rate Data

### Exercises in efficient collection and storage

------

December 29, 2017

-----

GOAL: Collect series from API once and store as one part of a SQL database. Store the date of the latest value and when it is more than X days old, start to check the API for a new datapoint. 


-----

To Do List:

 - Set up config file
 - Annotate existing code
 - Add code to retrieve full time series from API
 - Add code to store in SQL database
 - Move above two sections to area that only runs if no database or db empty.
 - Adjust code to retrieve latest datapoints 
 - Add code to save / replace latest datapoints in SQL db
 - Add code to check API only if X days from latest datapoint
 - Add code to generate note in SQL database of date when updated last
 - Annotate code
 - Clean up code
 

In [3]:
# Import python packages
import requests
print('requests v{}'.format(requests.__version__))
import pandas as pd
print('pandas v{}'.format(pd.__version__))
import sqlite3
print('sqlite3 v{}'.format(sqlite3.version))
import json

import datetime
today = '{:%Y-%m-%d}'.format(datetime.datetime.now())
print(today)

import config   # Local file config.py with BLS API key

requests v2.18.4
pandas v0.22.0
sqlite3 v2.6.0
2018-01-21


In [2]:
# BLS API - access to Bureau of Labor Statistics data
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(config.bls_key)

# Info for sending with post request
headers = {'Content-type': 'application/json'}
param = {"seriesid": ['LNS12300060'],  # BLS series ID
         "startyear": "2010", "endyear": "2016"}

# Request json data from BLS API
p = requests.post('{}{}'.format(url, key), 
                  data=json.dumps(param), 
                  headers=headers).json()

# Print whether API request was successful
print(p['status']) 

# Location within API results of time series and footnotes
r = p['Results']['series'][0]['data']

REQUEST_SUCCEEDED


In [3]:
# List comprehensions - clean json data and covert the 
# date, value, and footnote into pandas dataframe columns
dates = ['{} {}'.format(i['period'], i['year']) for i in r]
index = pd.to_datetime(dates)
data = {param['seriesid'][0]: [float(i['value']) for i in r]}
# .iloc[::-1] reverses the sorting/order of the dataframe
df = pd.DataFrame(index=index, data=data).iloc[::-1]

In [16]:
# Connect to SQL database
conn = sqlite3.connect('econ_data.db')

# Save dataframe to database
df.to_sql(param['seriesid'][0], conn, 
          if_exists='append', index_label='Date')

In [18]:
c = conn.cursor()
c.execute("select * from {} order by Date".format(param['seriesid'][0]))
print(c.fetchall())
c.close()
conn.commit()
conn.close()

[(u'2010-01-01 00:00:00', 75.1), (u'2010-01-01 00:00:00', 75.1), (u'2010-01-01 00:00:00', 75.1), (u'2010-02-01 00:00:00', 75.1), (u'2010-02-01 00:00:00', 75.1), (u'2010-02-01 00:00:00', 75.1), (u'2010-03-01 00:00:00', 75.1), (u'2010-03-01 00:00:00', 75.1), (u'2010-03-01 00:00:00', 75.1), (u'2010-04-01 00:00:00', 75.4), (u'2010-04-01 00:00:00', 75.4), (u'2010-04-01 00:00:00', 75.4), (u'2010-05-01 00:00:00', 75.1), (u'2010-05-01 00:00:00', 75.1), (u'2010-05-01 00:00:00', 75.1), (u'2010-06-01 00:00:00', 75.2), (u'2010-06-01 00:00:00', 75.2), (u'2010-06-01 00:00:00', 75.2), (u'2010-07-01 00:00:00', 75.1), (u'2010-07-01 00:00:00', 75.1), (u'2010-07-01 00:00:00', 75.1), (u'2010-08-01 00:00:00', 75.0), (u'2010-08-01 00:00:00', 75.0), (u'2010-08-01 00:00:00', 75.0), (u'2010-09-01 00:00:00', 75.1), (u'2010-09-01 00:00:00', 75.1), (u'2010-09-01 00:00:00', 75.1), (u'2010-10-01 00:00:00', 75.0), (u'2010-10-01 00:00:00', 75.0), (u'2010-10-01 00:00:00', 75.0), (u'2010-11-01 00:00:00', 74.8), (u'2010

In [11]:
c.fetchall()

ProgrammingError: Cannot operate on a closed cursor.

In [22]:
r[0]['period'][1:]

u'12'

In [6]:
# Retrieve data from SQL database
#pd.read_sql("select * from {}".format(param['seriesid'][0]), 
#            conn, index_col='Date', parse_dates=True).iloc[::-1].tail()